# European Weightlifting Championships ETL Pipeline (2019-2024)

**Author:** Senior Python Data Engineer  
**Objective:** Extract, Transform, and Load weightlifting championship data from Wikipedia into a unified, analysis-ready dataset.

---

## 1. Import Dependencies

In [1]:
# Standard library imports
import re
import warnings
from typing import Dict, List, Optional, Tuple

# Third-party imports
import pandas as pd
import plotly.graph_objects as go

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## 2. Configuration & Constants

In [2]:
# Year-to-URL mapping for all championships
YEAR_URLS: Dict[int, Dict[str, any]] = {
    2019: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2019',
        'male_table_idx': 5,
        'female_table_idx': 6
    },
    2020: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2020',
        'male_table_idx': 2,
        'female_table_idx': 3
    },
    2021: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2021',
        'male_table_idx': 2,
        'female_table_idx': 3
    },
    2022: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2022',
        'male_table_idx': 2,
        'female_table_idx': 3
    },
    2023: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2023',
        'male_table_idx': 2,
        'female_table_idx': 3
    },
    2024: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2024',
        'male_table_idx': 2,
        'female_table_idx': 3
    }
}

# Medal order for sorting
MEDAL_ORDER = {'Oro': 1, 'Plata': 2, 'Bronce': 3}

# Output file path
OUTPUT_FILE = 'processed_weightlifting_data_2019_2024.csv'

## 3. Data Extraction & Parsing Functions

In [3]:
def parse_athlete_data(cell_value: str) -> pd.Series:
    """
    Parse athlete data from Wikipedia table cell using regex patterns.
    
    Expected format: "Athlete Name Country 123 + 456 = 579"
    Where: 123 = Snatch, 456 = Clean&Jerk, 579 = Total
    
    Args:
        cell_value: Raw cell content from Wikipedia table
        
    Returns:
        pd.Series: [Nombre, Pais, Arrancada, DosTiempos, Total]
        Returns None values if parsing fails
    """
    # Handle missing/invalid data
    if pd.isna(cell_value):
        return pd.Series([None] * 5)
    
    try:
        # Clean invisible characters and footnotes
        cleaned = str(cell_value).replace('\u200b', '').split('[')[0].strip()
        
        # Regex pattern to extract three numbers (snatch + clean&jerk = total)
        # Pattern: captures "123 + 456 = 579" or "123 456 579"
        number_pattern = r'(\d+)\s*(?:\+|)\s*(\d+)\s*(?:=|)\s*(\d+)'
        numbers_match = re.search(number_pattern, cleaned)
        
        if not numbers_match:
            return pd.Series([None] * 5)
        
        snatch = int(numbers_match.group(1))
        clean_jerk = int(numbers_match.group(2))
        total = int(numbers_match.group(3))
        
        # Extract everything before the numbers
        text_before_numbers = cleaned[:numbers_match.start()].strip()
        tokens = text_before_numbers.split()
        
        if len(tokens) < 2:
            return pd.Series([None] * 5)
        
        # Country is typically the last token before numbers
        country = tokens[-1]
        
        # Athlete name is everything before the country
        athlete_name = ' '.join(tokens[:-1])
        
        return pd.Series([athlete_name, country, snatch, clean_jerk, total])
        
    except Exception as e:
        # Log the error but continue processing
        print(f"⚠️  Parsing error for value '{cell_value}': {str(e)}")
        return pd.Series([None] * 5)

In [4]:
def extract_medal_data(
    table: pd.DataFrame,
    medal_type: str,
    gender: str,
    year: int
) -> pd.DataFrame:
    """
    Extract and structure data for a specific medal type.
    
    Args:
        table: Raw DataFrame from Wikipedia
        medal_type: 'Oro', 'Plata', or 'Bronce'
        gender: 'M' or 'F'
        year: Championship year
        
    Returns:
        pd.DataFrame: Structured medal data
    """
    df = table[['Evento', medal_type]].copy()
    df['Medalla'] = medal_type
    
    # Parse athlete data
    df[['Nombre', 'Pais', 'Arrancada', 'DosTiempos', 'Total']] = \
        df[medal_type].apply(parse_athlete_data)
    
    # Add metadata
    df['Género'] = gender
    df['Año'] = year
    
    # Clean up
    df = df.drop(columns=[medal_type])
    
    return df

In [5]:
def process_championship_year(
    year: int,
    url: str,
    male_table_idx: int,
    female_table_idx: int
) -> pd.DataFrame:
    """
    Process all data for a single championship year.
    
    Args:
        year: Championship year
        url: Wikipedia URL for the championship
        male_table_idx: Index of male results table
        female_table_idx: Index of female results table
        
    Returns:
        pd.DataFrame: Combined and cleaned data for the year
    """
    print(f"\n{'='*60}")
    print(f"Processing Year: {year}")
    print(f"{'='*60}")
    
    try:
        # Fetch all tables from Wikipedia
        tables = pd.read_html(url)
        print(f"✓ Found {len(tables)} tables in the page")
        
        # Extract male data
        male_table = tables[male_table_idx].copy()
        male_table.columns = ['Evento', 'Oro', 'Plata', 'Bronce']
        
        male_oro = extract_medal_data(male_table, 'Oro', 'M', year)
        male_plata = extract_medal_data(male_table, 'Plata', 'M', year)
        male_bronce = extract_medal_data(male_table, 'Bronce', 'M', year)
        
        # Extract female data
        female_table = tables[female_table_idx].copy()
        female_table.columns = ['Evento', 'Oro', 'Plata', 'Bronce']
        
        female_oro = extract_medal_data(female_table, 'Oro', 'F', year)
        female_plata = extract_medal_data(female_table, 'Plata', 'F', year)
        female_bronce = extract_medal_data(female_table, 'Bronce', 'F', year)
        
        # Combine all data
        df_year = pd.concat([
            male_oro, male_plata, male_bronce,
            female_oro, female_plata, female_bronce
        ], ignore_index=True)
        
        # Data quality checks and cleaning
        initial_rows = len(df_year)
        
        # Drop rows with missing critical data
        df_year = df_year.dropna(
            subset=['Nombre', 'Pais', 'Arrancada', 'DosTiempos', 'Total']
        )
        
        # Fix country name inconsistencies
        df_year['Pais'] = df_year['Pais'].replace({'Unido': 'Reino Unido'})
        df_year['Nombre'] = df_year.apply(
            lambda row: row['Nombre'].replace(' Reino', '') 
            if row['Pais'] == 'Reino Unido' else row['Nombre'],
            axis=1
        )
        
        # Extract category and date from event
        df_year['Categoria'] = df_year['Evento'].str.extract(r'(\+?\d+\s?kg)')
        df_year['Fecha'] = df_year['Evento'].str.extract(r'\((\d{2}\.\d{2})\)')
        
        # Sort by medal priority and event
        df_year['OrdenMedalla'] = df_year['Medalla'].map(MEDAL_ORDER)
        df_year = df_year.sort_values(
            by=['OrdenMedalla', 'Evento']
        ).drop(columns='OrdenMedalla')
        
        df_year = df_year.reset_index(drop=True)
        
        dropped_rows = initial_rows - len(df_year)
        print(f"✓ Processed {len(df_year)} records ({dropped_rows} invalid rows dropped)")
        print(f"  - Male athletes: {len(df_year[df_year['Género'] == 'M'])}")
        print(f"  - Female athletes: {len(df_year[df_year['Género'] == 'F'])}")
        
        return df_year
        
    except Exception as e:
        print(f"❌ Error processing year {year}: {str(e)}")
        raise

## 4. ETL Pipeline Execution

In [6]:
# Process all championship years
all_years_data: List[pd.DataFrame] = []

for year, config in YEAR_URLS.items():
    df_year = process_championship_year(
        year=year,
        url=config['url'],
        male_table_idx=config['male_table_idx'],
        female_table_idx=config['female_table_idx']
    )
    all_years_data.append(df_year)

# Combine all years into unified dataset
df_all_years = pd.concat(all_years_data, ignore_index=True)

# Final sort by year and medal priority
df_all_years['OrdenMedalla'] = df_all_years['Medalla'].map(MEDAL_ORDER)
df_all_years = df_all_years.sort_values(
    by=['Año', 'OrdenMedalla', 'Evento']
).drop(columns='OrdenMedalla')
df_all_years = df_all_years.reset_index(drop=True)

print(f"\n{'='*60}")
print(f"ETL Pipeline Complete")
print(f"{'='*60}")
print(f"Total records: {len(df_all_years)}")
print(f"Years covered: {sorted(df_all_years['Año'].unique())}")
print(f"Countries: {df_all_years['Pais'].nunique()}")
print(f"Athletes: {df_all_years['Nombre'].nunique()}")

In [7]:
# Preview the unified dataset
print("\n📊 Dataset Preview:")
display(df_all_years.head(20))

print("\n📊 Dataset Info:")
display(df_all_years.info())

print("\n📊 Summary Statistics:")
display(df_all_years[['Arrancada', 'DosTiempos', 'Total']].describe())

## 5. Data Export

In [8]:
# Save unified dataset
df_all_years.to_csv(OUTPUT_FILE, index=False)
print(f"\n✅ Data saved to: {OUTPUT_FILE}")
print(f"   File size: {len(df_all_years)} rows × {len(df_all_years.columns)} columns")

## 6. Data Analysis & Visualization

### 6.1 Top 10 Countries by Medal Count

In [9]:
# Calculate medal counts by country
medallas_por_pais = df_all_years.groupby(['Pais', 'Medalla']).size().unstack(fill_value=0)
medallas_por_pais['Total'] = medallas_por_pais.sum(axis=1)
medallas_por_pais = medallas_por_pais.sort_values(by='Total', ascending=False)

print("Top 10 Countries by Total Medals:")
display(medallas_por_pais.head(10))

In [10]:
# Visualization: Top 10 Countries
top_10 = medallas_por_pais.head(10).drop(columns='Total')

fig = go.Figure()

for medalla in top_10.columns:
    fig.add_trace(go.Bar(
        name=medalla,
        x=top_10.index,
        y=top_10[medalla],
        hovertemplate='País: %{x}<br>Medallas: %{y}<extra></extra>'
    ))

fig.update_layout(
    barmode='stack',
    title='Top 10 Countries by Medal Count (2019–2024)',
    xaxis_title='Country',
    yaxis_title='Medal Count',
    legend_title='Medal Type',
    height=500,
    width=900
)

fig.show()
fig.write_image('chart_medallas.png', width=900, height=500)

### 6.2 Gender Equity Analysis - Medal Distribution

In [11]:
# Calculate gender distribution of medals
medallas_genero = df_all_years.groupby(['Pais', 'Género']).size().unstack(fill_value=0)

# Rename columns for clarity
if 'F' in medallas_genero.columns and 'M' in medallas_genero.columns:
    medallas_genero.columns = ['Femenino', 'Masculino']

# Calculate equity (smaller difference = more equitable)
medallas_genero['Diferencia'] = abs(
    medallas_genero['Femenino'] - medallas_genero['Masculino']
)
medallas_equidad = medallas_genero.sort_values(by='Diferencia')

print("Top 10 Most Equitable Countries (by medal distribution):")
display(medallas_equidad.head(10))

In [12]:
# Visualization: Gender Equity in Medals
top_equidad = medallas_equidad.head(10)[['Femenino', 'Masculino']]

fig = go.Figure()

for genero in top_equidad.columns:
    fig.add_trace(go.Bar(
        name=genero,
        x=top_equidad.index,
        y=top_equidad[genero],
        hovertemplate='País: %{x}<br>Medallas: %{y}<extra></extra>'
    ))

fig.update_layout(
    barmode='group',
    bargap=0.1,
    title='Top 10 Most Equitable Countries - Medal Distribution by Gender (2019–2024)',
    xaxis_title='Country',
    yaxis_title='Medal Count',
    legend_title='Gender',
    height=500,
    width=900
)

fig.show()
fig.write_image('chart_equidad_medallas.png', width=900, height=500)

### 6.3 Gender Equity Analysis - Performance Averages

In [13]:
# Calculate average total weight lifted by gender
promedios_total = df_all_years.groupby(['Pais', 'Género'])['Total'].mean().unstack()

# Rename columns
if 'F' in promedios_total.columns and 'M' in promedios_total.columns:
    promedios_total.columns = ['Femenino', 'Masculino']

# Calculate difference (gender performance gap)
promedios_total['Diferencia'] = abs(
    promedios_total['Femenino'] - promedios_total['Masculino']
)
promedios_equidad = promedios_total.sort_values(by='Diferencia')

print("Top 10 Countries with Most Similar Performance Between Genders:")
display(promedios_equidad.head(10))

In [14]:
# Visualization: Performance Equity
top_promedios = promedios_equidad.head(10)[['Femenino', 'Masculino']]

fig = go.Figure()

for genero in top_promedios.columns:
    fig.add_trace(go.Bar(
        name=genero,
        x=top_promedios.index,
        y=top_promedios[genero],
        hovertemplate='País: %{x}<br>Promedio: %{y:.1f} kg<extra></extra>'
    ))

fig.update_layout(
    barmode='group',
    bargap=0.1,
    title='Top 10 Countries - Average Total Weight Lifted by Gender (2019–2024)',
    xaxis_title='Country',
    yaxis_title='Average Total (kg)',
    legend_title='Gender',
    height=500,
    width=900
)

fig.show()
fig.write_image('chart_equidad_total.png', width=900, height=500)